In [1]:
import numpy as np  # probably don't need to load
import pandas as pd
#import datetime as dt
#import pandas_datareader.data as web  # probably don't need to load
#import quandl

#import blpapi
#from xbbg import blp
import CLOutilsPyXLL as clo

#import matplotlib.pyplot as plt
import seaborn as sns

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [2]:
path = 'Z:/Shared/Risk Management and Investment Technology/CLO Optimization/'
file = 'CLO17 portfolio as of 6.1.21.xlsm'
filepath= path+file
CLO_tab = 'CLO 17 Port as of 6.1'
Bid_tab = 'Bid.Ask 6.1'

# Create Model Portfolio Dataframe

In [3]:
import importlib
importlib.reload(clo)

<module 'CLOutilsPyXLL' from 'C:\\Users\\jknechtel\\Miniconda3\\RM-IT\\CLOutilsPyXLL.py'>

In [4]:
model_port = clo.create_model_port_df(filepath)

C:\Users\jknechtel\Miniconda3\RM-IT\CLOutilsPyXLL.py:233: RuntimeWarning: invalid value encountered in double_scalars
  apply(lambda x: 100*((x[0]*x[1]/100)+(x[2]*x[3]/100))/(x[0]+x[2]),axis=1) #I get runtime warnings often (prob div by 0)
C:\Users\jknechtel\Miniconda3\RM-IT\CLOutilsPyXLL.py:242: RuntimeWarning: invalid value encountered in double_scalars
  ((x[0]*x[5]/100+(x[6]+x[1])*x[2])/x[3])*100,axis=1 )  #I get runtime warnings often


skipped the zz_LXREP line drops


# Our problem

Credit to https://blog.thinknewfound.com/2018/08/trade-optimization/ for the inspiration. 
It isn't the same problem but the trade minimization part is exactly what we needed.  

The idea is you set the target (e.g. WAS) as a constraints in terms of the current (WAS)
and the amount for which you would like it to increase/decrease based on the trades. The objective
is then to reach that modified constraint while trying to minimize the # of trades.

$$\min_{\psi \in \mathbb{R}, y \in \{0,1\}} \sum_{i=0}^{n} (\psi_{i} + y_{i}) $$

\textbf{Subject to:}

$$ \psi_{i} \geq trade_{i} $$
$$ \psi_{i} \geq -trade_{i} $$
$$ \psi_{i} \leq A*y_{i} $$
$$ y_{i} \in \{0,1\} $$
$$ A \in \mathbb{R^+} $$

The above condition address specifically trade minimization.  It is equivalent to minimizing the
number of trades, but in LP format. Then we shall add the previously derived trade constraints. $A$ is 
a constant set to be greater than the largest trade, e.g. $ A = \max{(\textbf{trade})  +1} $.  It is also worth
noting that this formulation is specified in terms of weights.  That is

$$ trade_{i} = Par_{i}/\sum_{i=1}^{n} Par_{i} $$ 

To facilitate ease in writing the following conditions let's define, the Current Portfolio Total Par:
$$ \mathbb{P} = \sum_{i=1}^{n} Par_{i} $$

Each trade must be greater than amount in portfolio (no shorts) and up to the max amount tradeable.
Again we will specify in terms of weight as a % of the portfolio par.
$$ -CP_{i}/\mathbb{P} \leq trade_{i} \leq TradeLimit_{i}/\mathbb{P} $$

\textbf{Self-funding condition:} Slightly more restrictive specification (plus additional cash in any)
$$ \sum_{i=0}^{n} trade_{i} \times Bid_{i}/100 \leq 0 + CashtoSpend/\mathbb{P} $$
$$ \sum_{i=0}^{n} trade_{i} \times Ask_{i}/100 \leq 0 + CashtoSpend/\mathbb{P} $$

Here $Bid$ and $Ask$ are centered around 100 (i.e. 95-105, not .95-1.05)

\textbf{Par Burn Limit:} Slight more restrictive specification
$$ \sum_{i=0}^{n} trade_{i} \times (100 - Bid_{i}) \geq ParBLLim/\mathbb{P} $$

\textbf{Use of Cash:} This constraint is the least obvious one to me but required to force the
optimizer to use all the cash raised on trades.  It is likely a function of Par Burn being the most
restrictive constraint so the optimizer likes buying sub 100 loans which leaves cash left over.
$$ \sum_{i=0}^{n} trade_{i} \times (1+(100 - Bid_{i})/100) \leq 0 $$

\textbf{Weighted Average Stats constraints:} stated in comparison to the CP & the limit, with the 
$$ \sum_{i=0}^{n} trade_{i} \times RF_{i} \leq WARFTest - WARF_{CP} $$
$$ \sum_{i=0}^{n} trade_{i} \times RR_{i} \geq WARRTest - WARR_{CP} $$

\textbf{Loan type constraints:}  (% 2nd Lien, % Cov Lite, % CCC and less, etc) can be written like
$$ \sum_{i=0}^{n} trade_{i} \times Cov_{i} \leq [CovTest - Cov_{CP}] \times (1+ParBLLim/\mathbb{P}) $$
$$ \sum_{i=0}^{n} trade_{i} \times LienTwo_{i} \leq [LienTest - Lien_{CP}] \times (1+ParBLLim/\mathbb{P}) $$
$$ \sum_{i=0}^{n} trade_{i} \times CCC_{i} \leq [CCCTest - CCC_{CP}] \times (1+ParBLLim/\mathbb{P}) $$

where $Cov_{i}$ is an indicator that is 1 if the loan is Cov Lite, 0 otherwise.  All others likewise.

\textbf{Diversity constraint:} We can simplify it for now, but more to be done later on this one

$$ \sum_{i=0}^{n} MCDiversity_{i} \times t_{i} \geq Diversity_{CP} + DesiredAddLoss \geq DiversityTest $$

Note:  The Par burn condition and the self-funding conditions should be specified with the Bid on 
sales, and Ask on buys.  However the linear constraints are difficult, maybe impossible, to write
for the optimizer that way, so we either take one side (Buy) or can write with both sides which makes it more restrictive.

## Setup Variables

In [5]:
keyStats = ['Spread','Adj. WARF NEW','MC Div Score','Moodys Recovery Rate','S&P Recovery Rate (AAA)','Blended Price'] 
highLow = [1,-1,1,1,1,1]
weights = [0.285714286, 0.317460317, 0.238095238, 0, 0.158730159,0]
(np.array(weights)*np.array(highLow))
model_port = clo.desirability(model_port,keyStats,weights,highLow)

#currStats = [.0335,.0358,2772,2944,.482,.417,87,99.01,.038,.008,0.0,.015,.094,499303478]
keyConstraints = {'Spread':0.0315,
    'Adj. WARF NEW':2931,
    'WARF':2965,
    'MC Div Score':75,
    'Moodys Recovery Rate':0.43,
    'S&P Recovery Rate (AAA)':0.42,
    'Blended Price':0.1,
    'C_or_Less':0.1,
    'Lien':0.1,
    'Sub80':0.1,
    'Sub90':0.1,
    'CovLite':0.1}


otherConstraints = {'Cash to spend/raise':0,
    'Par Build(+) Loss(-) Limit':-2000000,
    'Max trade size (on buys)':1000000}


seeds = pd.DataFrame([1e6,1e6,0], index=['LX165896','LX185373','LX178941'])

array([ 0.28571429, -0.31746032,  0.23809524,  0.        ,  0.15873016,
        0.        ])

In [6]:
currStats = clo.Port_stats(model_port,weight_col='Par_no_default',format_output=False)
currStats = dict(zip(currStats.index,currStats.values))
currStats

{'Min Floating Spread Test - no Libor Floors': array([3.37017274]),
 'Min Floating Spread Test - With Libor Floors': array([3.61603092]),
 'Max Moodys Rating Factor Test (NEW WARF)': array([2772.15554151]),
 'Max Moodys Rating Factor Test (Orig WARF)': array([2923.89101183]),
 'Min Moodys Recovery Rate Test': array([48.07767825]),
 'Min S&P Recovery Rate Class A-1a': array([41.33893193]),
 'Moodys Diversity Test': array([86.4549]),
 'WAP': array([98.87454177]),
 'Percent C': array([3.91440179]),
 'Percent 2nd Lien': array([0.71657898]),
 'Percent Sub80': array([0.20173922]),
 'Percent Sub90': array([1.63585937]),
 'Percent CovLite': array([8.15311518]),
 'Total Portfolio Par (excl. Defaults)': array([5.04744485e+08])}

In [7]:
import pulp

In [8]:
solver_list = pulp.listSolvers()
solver_list

['GLPK_CMD',
 'PYGLPK',
 'CPLEX_CMD',
 'CPLEX_PY',
 'CPLEX_DLL',
 'GUROBI',
 'GUROBI_CMD',
 'MOSEK',
 'XPRESS',
 'PULP_CBC_CMD',
 'COIN_CMD',
 'COINMP_DLL',
 'CHOCO_CMD',
 'PULP_CHOCO_CMD',
 'MIPCL_CMD',
 'SCIP_CMD']

In [10]:
from pulp import *
probName = "Trade Minimization Problem"

trading_model = LpProblem(probName, LpMinimize)
t_vars = []
psi_vars = []
y_vars = []
A = 2

# Key Variables from our Constituent Universe
# turns the DF columns into an array, so be careful
# not to sort or modify the arrays or the indices
# won't line up.  I prefer to use the Dict way of
# setting constraints for this reason but this was
# simpler in this case.
CP = model_port['Current Portfolio'].to_numpy()
P = model_port['Current Portfolio'].to_numpy().sum()
Ask = model_port['Ask'].to_numpy()
Bid = model_port['Bid'].to_numpy()
LienTwo = model_port['Lien'].to_numpy()
CovLite = model_port['CovLite'].to_numpy()
SubCCC    = model_port['C_or_Less'].to_numpy()
SubEighty = model_port['Sub80'].to_numpy()
SubNinety = model_port['Sub90'].to_numpy()
D = model_port['Desirability'].to_numpy()
WARF = model_port['Adj. WARF NEW'].to_numpy()
WARR = model_port['S&P Recovery Rate (AAA)'].to_numpy()
#WAMRR = model_port['Moodys Recovery Rate'].to_numpy()
mcDiv = model_port['MC Div Score'].to_numpy()

CP = CP/P
n = len(CP)
tickers = model_port.index

# Constraint Limits
currStats = clo.Port_stats(model_port,weight_col='Par_no_default',format_output=False)
currStats = dict(zip(currStats.index,currStats.values))
#print(currStats)
    
# this would be better as a dictionary in case the sheet changes order
Cash_to_Spend = otherConstraints['Cash to spend/raise']
PBLim = otherConstraints['Par Build(+) Loss(-) Limit']
upperTradable = otherConstraints['Max trade size (on buys)']
#maxTrades = otherConstraints['Max # of new loans']   # not using atm
#print('Cash_to_Spend: ',Cash_to_Spend,' PBLim: ',PBLim,' upperTradable: ',upperTradable)

WARFTest = keyConstraints['Adj. WARF NEW']
WARFcp = currStats['Max Moodys Rating Factor Test (NEW WARF)'][0]
WARFdelta = WARFTest - WARFcp
RecoveryTest = keyConstraints['S&P Recovery Rate (AAA)']
RRcp = currStats['Min S&P Recovery Rate Class A-1a'][0]/100
RRdelta = RecoveryTest - RRcp
DiversityTest = -100
print('WARFTest: ',WARFTest,' WARFcp: ',WARFcp,' RecoveryTest: ',RecoveryTest,' RRcp: ',RRcp)
print('WARFdelta: ',WARFdelta,' RRdelta: ',RRdelta)

#TotalPar = currStats['Total Portfolio Par (excl. Defaults)'][0]
    #ParDenom = currStats[13]+PBLim # this is the lower limit of the new Par amt, use for % constraints
#SubC_Constr = keyConstraints['C_or_Less']*(P+PBLim) - currStats['Percent C'][0]*P/100
#Lien_Constr = keyConstraints['Lien']*(P+PBLim) - currStats['Percent 2nd Lien'][0]*P/100
#Sub80_Constr = keyConstraints['Sub80']*(P+PBLim) - currStats['Percent Sub80'][0]*P/100
#Sub90_Constr = keyConstraints['Sub90']*(P+PBLim) - currStats['Percent Sub90'][0]*P/100
#Cov_Constr = keyConstraints['CovLite']*(P+PBLim) - currStats['Percent CovLite'][0]*P/100

SubC_Constr = (keyConstraints['C_or_Less'] - currStats['Percent C'][0]/100)*(1+PBLim/P)
Lien_Constr = (keyConstraints['Lien'] - currStats['Percent 2nd Lien'][0]/100)*(1+PBLim/P)
Sub80_Constr = (keyConstraints['Sub80'] - currStats['Percent Sub80'][0]/100)*(1+PBLim/P)
Sub90_Constr = (keyConstraints['Sub90'] - currStats['Percent Sub90'][0]/100)*(1+PBLim/P)
Cov_Constr = (keyConstraints['CovLite'] - currStats['Percent CovLite'][0]/100)*(1+PBLim/P)

print('Lien: ',Lien_Constr,' Cov: ',Cov_Constr,' SubC: ',SubC_Constr,' Sub80: ',Sub80_Constr)

upperTradable = 1e6
# Convert to % weights instead of Par amounts for the solver
# we can multiply back through at the end


UB = CP.copy()
LB = CP.copy()
for k  in range(n):
    LB[k] = max(-CP[k],-upperTradable/P)  # no shorting and limited sell amt
    UB[k] = upperTradable/P
    
# seed trades should be set like x.lowBound = seedAmt, where x is the LXID variable (could set lb=ub=seedAmt)
# likewise loans to not buy x.upBound = 0, and to not sell x.lowBound = CP_i (or simply drop from DF)
#if ~seedTrades.isnull().values.all():
#    for i in seedTrades.index:
#        LB[i] = seedTrades.loc[i].values[0]
#        UB[i] = seedTrades.loc[i].values[0]

# Can't buy unAttractive loans
for i in np.where((model_port['Attractiveness']==1) | (model_port['Attractiveness']==2)):
    UB[i] = 0
        
# Can't sell very Attractive loans
#for i in model_port.loc[(model_port['Attractiveness']==5) ].index:
for i in np.where(model_port['Attractiveness']==5):
    LB[i] = 0
    
for i in range(n):
    t = LpVariable("t_" + str(i), LB[i], UB[i]) 
    t_vars.append(t)
    
    psi = LpVariable("psi_" + str(i), None, None)  # absolute value trick
    psi_vars.append(psi)
    
    y = LpVariable("y_" + str(i), 0, 1, LpInteger) #set y in {0, 1}, indicator trick
    y_vars.append(y)
    
# add our objective to minimize psi & y, which is the number of trades
trading_model += lpSum(psi_vars) + lpSum(y_vars), "Objective"
            
for i in range(n):
    trading_model += psi_vars[i] >= -t_vars[i]
    trading_model += psi_vars[i] >= t_vars[i]
    trading_model += psi_vars[i] <= A * y_vars[i]
    
#for i in range(n):
#    trading_model += phi_vars[i] >= -(w_diff[i] - t_vars[i])
#    trading_model += phi_vars[i] >= (w_diff[i] - t_vars[i])
    
# Make sure our trades sum to zero
trading_model += (lpSum(t_vars) <= 0 + Cash_to_Spend/P)

# this is where our constraints come in
# First the practical constraints are added to 'prob' (self-funding, parburn, etc)
trading_model += lpSum([ Bid[i]/100 * t_vars[i] for i in range(n)]) <= Cash_to_Spend/P , "Self-funding Bid"
    
trading_model += lpSum([ Ask[i]/100 * t_vars[i] for i in range(n)]) <= Cash_to_Spend/P , "Self-funding Ask"
    #prob += lpSum([ Mid[i]/100 * t for t, i in zip(trades,Trades)]) <= Cash_to_Spend , "Self-funding Mid"
# I think this needs to be Bid for CP and Ask for loan
trading_model += lpSum([((100-Bid[i])/100 * t_vars[i]) for i in range(n)]) >= PBLim/P, "Par Burn Limit"
    #prob += lpSum([((100-Mid[i])/100 * t) for t, i in zip(trades,Trades)]) >= PBLim, "Par Burn Limit"
# still kind of weird that this is needed, must be in corner solution
trading_model += lpSum([((1+(100-Bid[i])/100) * t_vars[i]) for i in range(n)]) >= 0, "Must use cash raised"
    #prob += lpSum([((1+(100-Mid[i])/100) * t) for t, i in zip(trades,Trades)]) >= 0, "Must use cash raised"

# then the Test Condition Hard constriants, WARF,RR, Div, etc
trading_model += lpSum([WARF[i] * t_vars[i] for i in range(n)]) <= WARFdelta, "WARF Test"
trading_model += lpSum([WARR[i] * t_vars[i] for i in range(n)]) >= RRdelta, "Recovery Test"
    
# need to derive a better representation of this constraint
trading_model += lpSum([mcDiv[i] * t_vars[i] for i in range(n)]) >= DiversityTest, "Diversity Test (simplified)"
    
    #
trading_model += lpSum([CovLite[i] * t_vars[i] for i in range(n)]) <= Cov_Constr, "Cov Test"
trading_model += lpSum([SubCCC[i] * t_vars[i] for i in range(n)]) <= SubC_Constr, "Sub C Test"
trading_model += lpSum([SubEighty[i] * t_vars[i] for i in range(n)]) <= Sub80_Constr, "Sub 80 Test"
trading_model += lpSum([SubNinety[i] * t_vars[i] for i in range(n)]) <= Sub90_Constr, "Sub 90 Test"
trading_model += lpSum([LienTwo[i] * t_vars[i] for i in range(n)]) <= Lien_Constr, "2nd Lien Test"

# The problem data is written to an .lp file
trading_model.writeLP(probName+".lp")

# Set our trade bounds
#trading_model += (lpSum(phi_vars) / 2. <= theta)
#trading_model.solve()
# The status of the solution is printed to the screen
print("Status:", LpStatus[trading_model.status])
    
results = pd.Series([t_i.value() for t_i in t_vars], index = tickers)
print ("Number of trades: " + str(sum([y_i.value() for y_i in y_vars])))

#print "Turnover distance: " + str((w_target - (w_old + results)).abs().sum() / 2.)


# Each of the variables is printed with it's resolved optimum value
# so this would be new portfolio and new to derive trades by comparing to old
#for v in trading_model.variables():
#        #print(v.name, "=", v.varValue)
#    model_port.loc[v.name,'NewPort'] = model_port.loc[v.name,'Par_no_default'] + v.varValue * \
#            (1+(100-model_port.loc[v.name,'Ask'])/100 if v.varValue > 0 else 
#            1+(100-model_port.loc[v.name,'Bid'])/100 )
#    model_port.loc[v.name,'CashDelta'] = v.varValue
#    model_port.loc[v.name,'Trade'] = 'Buy' if v.varValue > 0 else 'Sale' if v.varValue < 0 else np.nan

WARFTest:  2931  WARFcp:  2772.155541507291  RecoveryTest:  0.42  RRcp:  0.4133893193249341
WARFdelta:  158.84445849270878  RRdelta:  0.006610680675065872
Lien:  0.09246636380776041  Cov:  0.018395667258820837  SubC:  0.060614846296707874  Sub80:  0.09759436138185228


C:\Users\jknechtel\Miniconda3\lib\site-packages\pulp\pulp.py:1199: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


[psi_0,
 psi_1,
 psi_10,
 psi_100,
 psi_101,
 psi_102,
 psi_103,
 psi_104,
 psi_105,
 psi_106,
 psi_107,
 psi_108,
 psi_109,
 psi_11,
 psi_110,
 psi_111,
 psi_112,
 psi_113,
 psi_114,
 psi_115,
 psi_116,
 psi_117,
 psi_118,
 psi_119,
 psi_12,
 psi_120,
 psi_121,
 psi_122,
 psi_123,
 psi_124,
 psi_125,
 psi_126,
 psi_127,
 psi_128,
 psi_129,
 psi_13,
 psi_130,
 psi_131,
 psi_132,
 psi_133,
 psi_134,
 psi_135,
 psi_136,
 psi_137,
 psi_138,
 psi_139,
 psi_14,
 psi_140,
 psi_141,
 psi_142,
 psi_143,
 psi_144,
 psi_145,
 psi_146,
 psi_147,
 psi_148,
 psi_149,
 psi_15,
 psi_150,
 psi_151,
 psi_152,
 psi_153,
 psi_154,
 psi_155,
 psi_156,
 psi_157,
 psi_158,
 psi_159,
 psi_16,
 psi_160,
 psi_161,
 psi_162,
 psi_163,
 psi_164,
 psi_165,
 psi_166,
 psi_167,
 psi_168,
 psi_169,
 psi_17,
 psi_170,
 psi_171,
 psi_172,
 psi_173,
 psi_174,
 psi_175,
 psi_176,
 psi_177,
 psi_178,
 psi_179,
 psi_18,
 psi_180,
 psi_181,
 psi_182,
 psi_183,
 psi_184,
 psi_185,
 psi_186,
 psi_187,
 psi_188,
 psi_189,
 ps

Status: Not Solved


TypeError: unsupported operand type(s) for +: 'int' and 'NoneType'